參考網址[[link]](https://medium.com/%E8%8A%B1%E5%93%A5%E7%9A%84%E5%A5%87%E5%B9%BB%E6%97%85%E7%A8%8B/geocoding-%E6%89%B9%E9%87%8F%E8%99%95%E7%90%86%E5%9C%B0%E5%9D%80%E8%BD%89%E6%8F%9B%E7%B6%93%E7%B7%AF%E5%BA%A6-721ab2564c88)

# Define a Function

In [16]:
import requests
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
# from msedge.selenium_tools import EdgeOptions
# from msedge.selenium_tools import Edge
import random

# for Chrome (無介面執行)
options = webdriver.ChromeOptions()
options.add_argument("headless")

# # for Edge (無介面執行) (https://www.cnblogs.com/lixuelong/p/14451608.html)
# edge_options = EdgeOptions()
# edge_options.use_chromium = True
# # # 设置无界面模式，也可以添加其它设置
# edge_options.add_argument('headless')


In [ ]:
# 切資料(12份)


for i in range(12):
    dt_sub = dt.iloc[i:(i+1)*1000]
    dt_sub.to_csv(f'..\..\Roaming-Dogs_data\Location_data\Temple\宗教團體查詢結果_{i}.csv')


In [17]:
# addr 地址
# chromedriver.exe or msedgedriver.exe 的路徑

# for Chrome
path = 'C:\Program Files (x86)\Python-3.6.11\Python-3.6.11\chromedriver.exe'

# # for Edge
# path = 'C:\Program Files (x86)\Python-3.6.11\Python-3.6.11\msedgedriver.exe'

In [41]:
def get_coordinate(addr):
    
    # for Chrome 
    browser = webdriver.Chrome(executable_path = path ,options=options) #
        
    # # for Edge (https://www.cnblogs.com/lixuelong/p/14451608.html)     
    # browser = Edge(executable_path = path, options=edge_options) #
    
    browser.get("https://map.tgos.tw/TGOSCLOUD/Web/Map/TGOSViewer_Map.aspx") 
    
    wait = WebDriverWait(browser, 30)
    time.sleep(random.randint(0,2))  
    
    # 找到'定位'然後點下去
    wait.until(EC.presence_of_element_located((By.XPATH,"//*[@id='MenuTAB_content3']/div[9]"))).click()    
    
    
    #找到輸入地址的地方並且清空    
    wait.until(EC.presence_of_element_located((By.XPATH,"//*[@id='LocateBox_FunctionWork_Addr_Condition']")))
    search = browser.find_element_by_xpath("//*[@id='LocateBox_FunctionWork_Addr_Condition']")
    time.sleep(random.randint(1,2)) 
    search.clear()

    # 輸入地址 
    time.sleep(random.randint(1,2))
    search.send_keys(addr)
    
    # 按下'定位'！
    wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='LocateBox_FunctionWork_Addr_Query']"))).click() 
     
    # 選第一個地址
    # iframe = browser.find_element_by_xpath("//*[@id='LocateBox_FunctionWork_Addr_Reslut']/div")
    # wait.until(EC.frame_to_be_available_and_switch_to_it(iframe))
    wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='LocateBox_FunctionWork_Addr_Reslut']")))
    time.sleep(random.randint(1,2))
    browser.find_element_by_xpath("//*[@id='LocateBox_FunctionWork_Addr_Reslut']/div[1]").click()

    # 轉到跳出來的框框(用 xpath 比較好喔！)
   
    try:
        xpath = "//*[@id='MapBox']/div[1]/div[2]/div/p[1]"  
        infomation = browser.find_element_by_xpath(xpath).text
        coor = infomation.split('\n')
        full_addr = coor[0].split("：")[-1]
        X = coor[1].split("：")[-1]
        Y = coor[2].split("：")[-1]
        browser.quit()  
        print([full_addr, X, Y])
        return [full_addr, X, Y]

    except NoSuchElementException:
        browser.quit()  
        print([addr, 0, 0])
        return [addr, 0, 0]      

In [45]:
print(get_coordinate('高雄市左營大路572號'))

['高雄市左營區尾北里2鄰左營大路572號', '177711.366', '2510168.102']


# 讀入表格

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

讀資料

In [ ]:

# path_market = '..\..\Roaming-Dogs_data\Location_data\Temple\宗教團體查詢結果.csv'
# dt = pd.read_csv(path_market)
# Counter(dt['縣市']).most_common()

'''
[('臺南市', 1610), ('高雄市', 1421), ('屏東縣', 1122), ('新北市', 923), ('臺中市', 901), ('彰化縣', 855), ('雲林縣', 800),
 ('嘉義縣', 705), ('宜蘭縣', 654), ('南投縣', 471), ('苗栗縣', 366), ('桃園市', 279), ('基隆市', 221), ('臺東縣', 214),
 ('臺北市', 212), ('金門縣', 190), ('澎湖縣', 186), ('新竹縣', 184), ('花蓮縣', 177), ('新竹市', 162), ('嘉義市', 122),
 ('連江縣', 67)]
 '''


轉經緯度(拜託讓我轉！)

In [ ]:
lon_lat = []
print(dt.shape[0])

for i in range(dt.shape[0]):

    print(i)            
    address = dt['地址.1'][i]
    x = get_coordinate(address)
    lon_lat.append(x) 
    time.sleep(random.randint(10,15))

In [ ]:
lon_lat_df = pd.DataFrame(lon_lat)
lon_lat_df.columns = ['完整地址', '經度', '緯度']
print(lon_lat_df.shape)

In [ ]:
# encoding 記得轉，不然是亂碼！
lon_lat_df.to_csv(f'..\..\Roaming-Dogs_data\Location_data\Temple\寺廟_{c}_經緯度_01.csv', encoding='big5', index = None)